In [140]:
using Random
using LinearAlgebra
using PyCall
open3d = pyimport("open3d")

PyObject <module 'open3d' from '/Users/yanyanggu/.julia/conda/3/aarch64/lib/python3.12/site-packages/open3d/__init__.py'>

In [141]:
function find_plane(pts, maxIteration, thresh)
    n_points = size(pts, 1)
    best_eq = []
    best_inliers = []

    for it in 1:maxIteration
        # Samples 3 random points
        id_samples = rand(1:n_points, 3)
        pt_samples = pts[id_samples, :]

        # Define two vectors on the plane
        vecA = pt_samples[2, :] - pt_samples[1, :]
        vecB = pt_samples[3, :] - pt_samples[1, :]

        # Cross product to find the normal vector of the plane
        vecC = cross(vecA, vecB)
        vecC /= norm(vecC)  # Normalize the vector

        # Compute the plane equation: vecC[1]*x + vecC[2]*y + vecC[3]*z = -k
        k = -dot(vecC, pt_samples[2, :])
        plane_eq = [vecC[1], vecC[2], vecC[3], k]

        # Calculate distances from each point to the plane
        dist_pt = (plane_eq[1] * pts[:, 1] .+ plane_eq[2] * pts[:, 2] .+ plane_eq[3] * pts[:, 3] .+ plane_eq[4]) /
                  sqrt(plane_eq[1]^2 + plane_eq[2]^2 + plane_eq[3]^2)

        # Select inliers within the threshold
        pt_id_inliers = findall(abs.(dist_pt) .<= thresh)
        
        # Check if this is the best model so far
        if length(pt_id_inliers) > length(best_inliers)
            if plane_eq[3]<0.00001
                best_eq = plane_eq
                best_inliers = pt_id_inliers
            end
        end
    end

    return best_eq, best_inliers
end

find_plane (generic function with 1 method)

In [142]:
println(ispath("Downloads/TLS_kitchen.ply"))
file_path = "Downloads/TLS_kitchen.ply"    # Replace with your PLY file path

# Load a PLY file
mesh = open3d.io.read_point_cloud(file_path)

# Visualize the mesh
open3d.visualization.draw_geometries([mesh])

true


In [143]:
points = convert(Array{Float32}, mesh.points)
plane, inliers = find_plane(points, 1000, 0.1)
allindex = 1:size(points,1)
outliers = setdiff(allindex, inliers)

369451-element Vector{Int64}:
      1
      2
      3
      4
      6
      7
      8
      9
     10
     11
     12
     14
     15
      ⋮
 511015
 511016
 511017
 511018
 511019
 511020
 511021
 511022
 511023
 511024
 511025
 511026

In [144]:
pcd = open3d.geometry.PointCloud()
pcd.points = pycall(open3d.utility.Vector3dVector, PyObject, points[inliers,:])
single_color = [1.0, 0.0, 0.0]  # Red color
colors = [single_color for _ in 1:size(inliers, 1)]  # Assign the same color to all points
# Set the colors to the point cloud
pcd.colors = pycall(open3d.utility.Vector3dVector, PyObject, colors)

pcd_o = open3d.geometry.PointCloud()
pcd_o.points = pycall(open3d.utility.Vector3dVector, PyObject, points[outliers,:])
single_color = [0.0, 0.0, 1.0]  # Red color
colors = [single_color for _ in 1:size(outliers, 1)]  # Assign the same color to all points
# Set the colors to the point cloud
pcd_o.colors = pycall(open3d.utility.Vector3dVector, PyObject, colors)

open3d.visualization.draw_geometries([pcd, pcd_o])